# Abalone Age Prediction by Multi-class Classification using ANN (testing)

## Load dataset for the training process

In [1]:
import numpy as np
import pandas as pd

### Open the CSV

In [2]:
df_test = pd.read_csv("abalone_test.csv")
print("Num test:", len(df_test))

Num test: 835


### Create dataset for testing process

In [3]:
import tensorflow as tf
import tensorflow_addons as tfa

2023-05-12 14:01:29.815716: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-12 14:01:29.817303: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 14:01:29.857204: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 14:01:29.857937: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 14:01:30.492640: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

### Features to be used and target

In [4]:
x_names = ["Sex", "Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight"]
y_name = "Rings"
y_classes = list(range(1, 30)) # 1 to 29 (from EDA)

### Convert Pandas dataframe to TF dataset

In [5]:
def df_to_dataset(df: pd.DataFrame) -> tuple:
    features = df[x_names].to_numpy(dtype=np.float32)
    labels = df[[y_name]].to_numpy(dtype=np.int64).squeeze(axis=1)
    labels = np.clip(labels, min(y_classes), max(y_classes))  # limit the age range to 1 to 29
    labels = labels - 1  # map class #0 to Rings 1
    labels = tf.keras.utils.to_categorical(labels, num_classes=len(y_classes))  # convert integer labels to one-hot encoding
    return features, labels

In [6]:
feat_test, lab_test = df_to_dataset(df_test)

## Model testing

In [7]:
from tqdm.notebook import tqdm

### Which device we will use for training process (CPU/GPU)

If a TensorFlow operation has both CPU and GPU implementations, by default, the GPU device is prioritized when the operation is assigned. So, no need to manually specify here.

*Reference: [Use a GPU](https://www.tensorflow.org/guide/gpu)*

In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2023-05-12 14:01:31.586221: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-12 14:01:31.586487: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Create the model

In [9]:
import nets

model = nets.get_model_mlp(n_classes=len(y_classes))

### Load the trained weights of the model parameters

In [10]:
WEIGHT_PATH = "runs_clf/train_20230512_132808/best"
model.load_weights(WEIGHT_PATH)

### Prepare the logger

In [11]:
# metrics to log
metrics_l = [
        tf.keras.metrics.CategoricalAccuracy(),
        tfa.metrics.F1Score(
            num_classes=len(y_classes),
            average="weighted",
            name="weighted_f1")
    ]

### The testing process

In [12]:
# configure model
model.compile(metrics=metrics_l)

In [13]:
BATCH_SIZE = 64
_ = model.evaluate(
    feat_test,
    lab_test,
    batch_size=BATCH_SIZE,
    verbose=1)
# use `model.predict` if the data has no label, i.e., predicting the label of new data

14/14 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - categorical_accuracy: 0.2503 - weighted_f1: 0.2361
